In [1]:
import anndata
import scanpy as sc
import scvi
import h5py
import scipy
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

Global seed set to 0


In [3]:
ad_atac = anndata.read_h5ad('/home/yuanh/sc_basset/datasets/10x_ARC_mouse_brain/ad_atac.h5ad')
ad_rna = anndata.read_h5ad('/home/yuanh/sc_basset/datasets/10x_ARC_mouse_brain/scvi/ad_rna_scvi.h5ad')
ad_atac.obs['leiden'] = ad_rna.obs['leiden'].values

def shared_neighbor_metrics(ad, n_neighbors, m_RNA_neighbors, label_obs_idx):
    # compute scATAC nearest neighbor graph
    sc.pp.neighbors(ad, n_neighbors=n_neighbors+1, use_rep='projection') # n nearest neighbor of each cell
    m_ATAC_neighbors = [i.indices for i in ad.obsp['distances']] # neighbor idx for each cell
    
    # metrics
    neighbor_label = ad.obs.iloc[np.concatenate(m_ATAC_neighbors, axis=0), label_obs_idx] # label for all neighbors
    cell_label = ad.obs.iloc[np.repeat(np.arange(len(m_ATAC_neighbors)), [len(j) for j in m_ATAC_neighbors]), label_obs_idx] # label for all cells
    n_shared_labels = (neighbor_label.values==cell_label.values).sum() / len(m_ATAC_neighbors)
    n_shared_neighbors = np.mean([len(np.intersect1d(i,j)) for i,j in zip(m_RNA_neighbors, m_ATAC_neighbors)])
    return n_shared_labels, n_shared_neighbors


In [4]:
pvi = scvi.model.PEAKVI.load("trained_model", ad_atac)
latent = pvi.get_latent_representation()
pd.DataFrame(latent).to_csv('projection.csv')

ad_atac.obsm["projection"] = latent
sc.pp.neighbors(ad_atac, use_rep="projection")
sc.tl.umap(ad_atac)
ad_atac.write('ad_scvi.h5ad')

INFO     .obs[_scvi_batch] not found in target, assuming every cell is same category         
INFO     .obs[_scvi_labels] not found in target, assuming every cell is same category        
INFO     Using data from adata.X                                                             


/home/yuanh/my_envs/scvi/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function transfer_anndata_setup is deprecated; This method will be removed in 0.15.0. Please avoid building any new dependencies on it.
  warnings.warn(msg, category=FutureWarning)


INFO     Registered keys:['X', 'batch_indices', 'labels']                                    
INFO     Successfully registered anndata object containing 4881 cells, 37647 vars, 1 batches,
         1 labels, and 0 proteins. Also registered 0 extra categorical covariates and 0 extra
         continuous covariates.                                                              
